---
title: "End-to-End Benchmarking"
---

## Processing benchmark results

### Import dependencies

The CarbonPlan team put together some utilities for parsing, processing, and visualizing the benchmarking results in [carbonplan_benchmarks](https://github.com/carbonplan/benchmark-maps). We'll use those utilities along with the [Holoviz](https://holoviz.org/) HoloViz suite of tools for visualization to show some individual benchmarking results.

In [1]:
import carbonplan_benchmarks.analysis as cba
import holoviews as hv
import hvplot.pandas
import pandas as pd

pd.options.plotting.backend = "holoviews"

### Show individual results

Define URIs for the baseline images and metadata

In [2]:
baseline_fp = "s3://carbonplan-benchmarks/benchmark-data/v0.2/baselines.json"
url_filter = "carbonplan-benchmarks.s3.us-west-2.amazonaws.com/data/"
metadata_fp = (
    "s3://carbonplan-benchmarks/benchmark-data/v0.2/data-2023-08-24T00-33-44.json"
)

Define some plot options

In [3]:
plt_opts = {"width": 500, "height": 200}

Load the data and baseline images

In [4]:
metadata, trace_events = cba.load_data(metadata_path=metadata_fp, run=0)
snapshots = cba.load_snapshots(snapshot_path=baseline_fp)

Process the data

In [5]:
data = cba.process_run(
    metadata=metadata, trace_events=trace_events, snapshots=snapshots
)

Create summary statistics

In [6]:
cba.create_summary(metadata=metadata, data=data, url_filter=url_filter)

,action,approach,benchmark_version,browser_name,browser_version,headless,playwright_python_version,provider,run_number,timeout,...,filtered_requests_average_encoded_data_length,filtered_requests_maximum_encoded_data_length,zoom,duration,min_rmse,fps,request_duration,request_percent,non_request_duration,actual_chunk_size
dataset,,,,,,,,,,,,,,,,,,,,,
pyramids-v2-3857-True-256-1-0-0-f4-0-0-0-gzipL1-100,zoom_in,dynamic-client,v0.2,chromium,116.0.5845.82,True,1.37.0,aws,1,False,...,63488.00,125345.0,0.0,2313.558,0.0,59.648386,408.764,17.668198,1904.794,0.524288
pyramids-v2-3857-True-256-1-0-0-f4-0-0-0-gzipL1-100,zoom_in,dynamic-client,v0.2,chromium,116.0.5845.82,True,1.37.0,aws,1,False,...,111303.25,206084.0,1.0,2468.501,0.0,60.360518,1365.362,55.311381,1103.139,0.524288
pyramids-v2-3857-True-256-1-0-0-f4-0-0-0-gzipL1-100,zoom_in,dynamic-client,v0.2,chromium,116.0.5845.82,True,1.37.0,aws,1,False,...,157726.00,342805.0,2.0,1591.088,0.0,60.336072,987.093,62.038869,603.995,0.524288
pyramids-v2-3857-True-256-1-0-0-f4-0-0-0-gzipL1-100,zoom_in,dynamic-client,v0.2,chromium,116.0.5845.82,True,1.37.0,aws,1,False,...,164329.00,334586.0,3.0,1229.950,0.0,59.352006,593.159,48.226269,636.791,0.524288


Plot the requests, frames, and baseline comparisons

In [7]:
requests_plt = cba.plot_requests(data["request_data"], url_filter=url_filter).opts(
    **plt_opts
)
frames_plt = cba.plot_frames(data["frames_data"], yl=2.5).opts(**plt_opts)
rmse_plt = cba.plot_screenshot_rmse(
    screenshot_data=data["screenshot_data"], metadata=metadata
).opts(**plt_opts)
(requests_plt + frames_plt + rmse_plt).cols(1)

/Users/max/mambaforge/envs/benchmark-maps/lib/python3.10/site-packages/holoviews/plotting/bokeh/plot.py:987: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  layout_plot = gridplot(
/Users/max/mambaforge/envs/benchmark-maps/lib/python3.10/site-packages/holoviews/plotting/bokeh/plot.py:987: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  layout_plot = gridplot(


:Layout
   .Overlay.I   :Overlay
      .Rectangles.I :Rectangles   [x0,y0,x1,y1]
      .Curve.I      :Curve   [time]   (y,encoded_data_length,url,method,total_response_time_ms,request_start,response_end)
      .Curve.II     :Curve   [time]   (y,encoded_data_length,url,method,total_response_time_ms,request_start,response_end)
      .Curve.III    :Curve   [time]   (y,encoded_data_length,url,method,total_response_time_ms,request_start,response_end)
      .Curve.IV     :Curve   [time]   (y,encoded_data_length,url,method,total_response_time_ms,request_start,response_end)
      .Curve.V      :Curve   [time]   (y,encoded_data_length,url,method,total_response_time_ms,request_start,response_end)
      .Curve.VI     :Curve   [time]   (y,encoded_data_length,url,method,total_response_time_ms,request_start,response_end)
      .Curve.VII    :Curve   [time]   (y,encoded_data_length,url,method,total_response_time_ms,request_start,response_end)
      .Curve.VIII   :Curve   [time]   (y,encoded_data_length,url,method,total_response_time_ms,request_start,response_end)
      .Curve.IX     :Curve   [time]   (y,encoded_data_length,url,method,total_response_time_ms,request_start,response_end)
      .Curve.X      :Curve   [time]   (y,encoded_data_length,url,method,total_response_time_ms,request_start,response_end)
      .Curve.XI     :Curve   [time]   (y,encoded_data_length,url,method,total_response_time_ms,request_start,response_end)
      .Curve.XII    :Curve   [time]   (y,encoded_data_length,url,method,total_response_time_ms,request_start,response_end)
      .Curve.XIII   :Curve   [time]   (y,encoded_data_length,url,method,total_response_time_ms,request_start,response_end)
      .Curve.XIV    :Curve   [time]   (y,encoded_data_length,url,method,total_response_time_ms,request_start,response_end)
      .Curve.XV     :Curve   [time]   (y,encoded_data_length,url,method,total_response_time_ms,request_start,response_end)
   .Overlay.II  :Overlay
      .Rectangles.I   :Rectangles   [x0,y0,x1,y1]
      .Rectangles.II  :Rectangles   [x0,y0,x1,y1]
      .Rectangles.III :Rectangles   [x0,y0,x1,y1]
      .Rectangles.IV  :Rectangles   [x0,y0,x1,y1]
   .NdOverlay.I :NdOverlay   [Variable]
      :Curve   [startTime]   (value)